<a href="https://colab.research.google.com/github/priyank-1/Deep_Learning/blob/main/Q3_DPL_ASSIGNMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install scikeras
# !pip install --upgrade python

# !pip install keras
# !pip install imblearn
# !pip install imbalanced-learn
# !pip install --upgrade scikit-learn
# !pip install --upgrade keras
# import sklearn
# print(sklearn.__version__)


In [1]:
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from imblearn.pipeline import Pipeline
from sklearn.utils import resample
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import time
import numpy as np

In [2]:
data = load_breast_cancer()
scaler = StandardScaler()
x = data.data
y = data.target
x = scaler.fit_transform(x)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3 ,random_state = 34)

In [3]:
oversampler = SMOTE()
x_train_re ,y_train_re = oversampler.fit_resample(x_train,y_train)
undersampler = RandomUnderSampler()
x_train_re, y_train_re = undersampler.fit_resample(x_train_re, y_train_re)
scaler = StandardScaler()
x_train_sc = scaler.fit_transform(x_train_re)
x_test_sc = scaler.transform(x_test)


np.isnan(y_train_re).sum()

0

In [4]:
mlp = MLPClassifier(max_iter=1000, early_stopping=True, validation_fraction=0.1, tol=1e-4, n_iter_no_change=10)

In [5]:
param_grid = {
    'hidden_layer_sizes': [(30,), (30,60,), (25, 50,15)],
    'activation': ['relu', 'tanh'],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'batch_size': [32, 64, 128]
}

In [6]:
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, cv=5 ,n_jobs = -1)
start_time = time.time()
grid_search.fit(x_train_sc, y_train_re)
end_time = time.time()

print("Time Take to train seconds. ",format(end_time - start_time))

Time Take to train seconds.  18.681212425231934


In [7]:
best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)

Best hyperparameters: {'activation': 'relu', 'batch_size': 32, 'hidden_layer_sizes': (30, 60), 'learning_rate_init': 0.01}


In [10]:
final_mlp = MLPClassifier(**best_params)
final_mlp.fit(x_train_sc , y_train_re)

MLPClassifier(batch_size=32, hidden_layer_sizes=(30, 60),
              learning_rate_init=0.01)

In [11]:
test_accuracy = final_mlp.score(x_test_sc,y_test)
print(test_accuracy)

0.9824561403508771


# WITH CPU AS RUNTIME ENV
**Time Take to train seconds.  28.231532096862793**

**Best hyperparameters: {'activation': 'relu', 'batch_size': 64, 'hidden_layer_sizes': (30, 60), 'learning_rate_init': 0.01}**

**Accuracy = 0.9766081871345029**


******

# WITH GPU AS RUNTIME ENV
**Time Take to train seconds.  18.6812124252319343**

**Best hyperparameters: {'activation': 'relu', 'batch_size': 32, 'hidden_layer_sizes': (30, 60), 'learning_rate_init': 0.01}**

**Accuracy = 0.9824561403508771**
